In [3]:
# Using TensorFlow to minimize a polynomial, 2 x squared plus 5 x plus 4

import tensorflow as tf

opt = tf.optimizers.Adam(learning_rate=0.001)

x = tf.Variable(10.0, trainable=True)
with tf.GradientTape() as t:
    # no need to watch a variable:
    # trainable variables are always watched
    log_x = tf.math.log(x)
    y = tf.math.square(log_x)

# Is the tape that computes the gradients!
trainable_variables = [x]
gradients = t.gradient(y, trainable_variables)
# The optimizer applies the update, using the variables
# and the optimizer update rule
opt.apply_gradients(zip(gradients, trainable_variables))
        
# The correct minimium is at x=1.25, with a value of 0.875.  It takes about 25 iterations to reach this.


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>